# Unit Commitment

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.7.post55.dev0+g8115b0f'

In [3]:
ams.config_logger(stream_level=10)

In [4]:
case14 = ams.get_case('ieee14/ieee14_uced.xlsx')
case39 = ams.get_case('ieee39/ieee39_uced.xlsx')

sp0 = ams.load(case14, setup=True)
sp = ams.load(case39, setup=True)

Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee14/ieee14_uced.xlsx"...
Input file parsed in 0.1519 seconds.
Adjust bus index to start from 0.
System set up in 0.0030 seconds.
Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/ieee39/ieee39_uced.xlsx"...
Input file parsed in 0.0523 seconds.
Adjust bus index to start from 0.
System set up in 0.0044 seconds.


In [5]:
stg = pd.DataFrame()
stg['idx'] = sp.StaticGen.get(src='idx', attr='v', idx=sp.StaticGen.get_idx())
stg['zone'] = sp.StaticGen.get(src='zone', attr='v', idx=sp.StaticGen.get_idx())
stg['Sn'] = sp.StaticGen.get(src='Sn', attr='v', idx=sp.StaticGen.get_idx())
stg

,idx,zone,Sn
0,PV_30,ZONE2,1040.0
1,PV_32,ZONE1,725.0
2,PV_33,ZONE3,652.0
3,PV_34,ZONE3,508.0
4,PV_35,ZONE3,687.0
5,PV_36,ZONE3,580.0
6,PV_37,ZONE2,564.0
7,PV_38,ZONE3,865.0
8,PV_39,ZONE1,1100.0
9,Slack_31,ZONE1,1500.0


In [6]:
sp.RTED.setup()

RTED data check passed.
- Generating symbols for RTED
Set constrs pb: sum(pd) - sum(pg) == 0
Set constrs pinj: Cg@(pn - pd) - pg == 0
Set constrs lub: PTDF @ (pn - pd) - rate_a <= 0
Set constrs llb: - PTDF @ (pn - pd) - rate_a <= 0
Set constrs rbu: gs @ pru - du == 0
Set constrs rbd: gs @ prd - dd == 0
Set constrs rru: pg + pru - pmax <= 0
Set constrs rrd: -pg + prd - pmin <= 0
Set constrs rgu: pg - pg0 - R10h <= 0
Set constrs rgd: -pg + pg0 - R10h <= 0
RTED model set up in 0.0090 seconds.


True

In [7]:
sp.RTED.run(solver='GUROBI', reoptimize=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-21


RTED solved as optimal in 0.0268 seconds with exit code 0.


True

In [8]:
sp.ED.run()

Setup model of ED
ED data check passed.
- Generating symbols for ED
Set constrs pb: pds - Spg @ pg == 0
Set constrs pinj: Cg @ (pn - pdR) - pg == 0
Set constrs lub: PTDF @ (pn - pdR) - RAr@cdup <= 0
Set constrs llb: -PTDF @ (pn - pdR) - RAr@cdup <= 0
Set constrs rgu: pg @ Mr - RR30 <= 0
Set constrs rgd: -pg @ Mr - RR30 <= 0
Set constrs rgu0: pg[:, 0] - pg0 - R30 <= 0
Set constrs rgd0: - pg[:, 0] + pg0 - R30 <= 0
ED model set up in 0.0076 seconds.
ED solved as optimal in 0.0542 seconds with exit code 0.


True

In [9]:
# FIXME: Non-spinning reserve demand will result into infeasibility

sp.NSR.demand.v = np.array([0., 0, 0])

In [10]:
sp.UC.run(solver='GUROBI', reoptimize=True)

Setup model of UC
UC data check passed.
- Generating symbols for UC
Set constrs pb: pds - Spg @ zug == 0
Set constrs pinj: Cg @ (pn - pdR) - pg == 0
Set constrs lub: PTDF @ (pn - pdR) - RAr@cdup <= 0
Set constrs llb: -PTDF @ (pn - pdR) - RAr@cdup <= 0
Set constrs rgu: pg @ Mr - RR30 <= 0
Set constrs rgd: -pg @ Mr - RR30 <= 0
Set constrs rgu0: pg[:, 0] - pg0 - R30 <= 0
Set constrs rgd0: - pg[:, 0] + pg0 - R30 <= 0
Set constrs actv: ugd @ Mr - vgd[:, 1:] <= 0
Set constrs actv0: ugd[:, 0] - ug0  - vgd[:, 0] <= 0
Set constrs actw: -ugd @ Mr - wgd[:, 1:] <= 0
Set constrs actw0: -ugd[:, 0] + ug0 - wgd[:, 0] <= 0
Set constrs zuglb: - zug + pg <= 0
Set constrs zugub: zug - pg - Mzug[0] * (1 - ugd) <= 0
Set constrs zugub2: zug - Mzug[0] * ugd <= 0
Set constrs nsr: gs@(-pg + zug) + Rdnsr <= 0
Set constrs sr: gs@(zug - multiply(Rpmax, ugd)) + Rdsr <= 0
UC model set up in 0.0183 seconds.
UC solved as optimal in 0.1724 seconds with exit code 0.


True

In [11]:
rtn = sp.UC

# spinning reserve
np.matmul(rtn.gs.v, (rtn.zug.v - np.multiply(rtn.Rpmax.v, rtn.ugd.v))) + rtn.Rdsr.v

# non-spinning reserve
np.matmul(rtn.gs.v, (- rtn.pg.v + rtn.zug.v)) + rtn.Rdnsr.v

array([[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])